In [ ]:
import os
import keras
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from keras.models import Model
import matplotlib.pyplot as plt
from keras import layers, models
from keras.utils import plot_model
from keras.models import load_model
from keras.models import Sequential
from skimage.transform import resize
from keras.regularizers import l1_l2
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.optimizers import Adamax,Nadam,Adam,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization,Input, concatenate, AveragePooling2D,GlobalAveragePooling2D,ReLU, Add
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Unzip the dataset
!unzip train.zip

Archive:  train.zip
  inflating: train_data.npz          


In [ ]:
#Unzip the dataset
!unzip test-11.zip

Archive:  test-11.zip
  inflating: test_data.npz           


In [ ]:
data = np.load('train_data.npz')
test=np.load("test_data.npz")
X_loaded = data['X_train']
y_loaded = data['y_train']
index=test['index']
test_images = test['X_test']

In [ ]:
print("Training Images shape : ",X_loaded.shape)
print("Test Images shape : ",test_images.shape)

Training Images shape :  (8800, 164, 164, 3)
Test Images shape :  (2200, 164, 164, 3)


In [ ]:
# Define the new size
new_size = (128, 128, 3)

In [ ]:
# Resize each image in X_loaded to new_size
X_resized = np.array([resize(img, new_size) for img in X_loaded])

In [ ]:
# Resize each image in test_images to new_size
X_resized_test = np.array([resize(img, new_size) for img in test_images])

In [ ]:
print("Training Images shape after resize : ",X_resized.shape)
print("Test Images shape after resize : ",X_resized_test.shape)

Training Images shape after resize :  (8800, 128, 128, 3)
Test Images shape after resize :  (2200, 128, 128, 3)


In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_resized, y_loaded, test_size=0.15, random_state=43)

In [ ]:
print(f"Resized {X_resized.shape[0]} images to {new_size}.")
print(f"Training set: {X_train.shape}, Validation set: {X_validation.shape}")

Resized 8800 images to (128, 128, 3).
Training set: (7480, 128, 128, 3), Validation set: (1320, 128, 128, 3)


In [ ]:
# Caluate mean and subtract for normalization
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_validation -= mean_image
X_resized_test -= mean_image

In [ ]:
num_classes = len(np.unique(y_loaded))
print(f"Number of classes: {num_classes}")

Number of classes: 11


In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_validation = keras.utils.to_categorical(y_validation, num_classes)

In [ ]:
def residual_block(x, filters, strides=(1, 1), use_projection=False):
    """ Defines a residual block in a simplified ResNet-like model """
    shortcut = x

    # Convolutional layers
    x = layers.Conv2D(filters=filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Projection shortcut to match dimensions
    if use_projection:
        shortcut = layers.Conv2D(filters=filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
        x = layers.Dropout(0.3)(x)

    # Add shortcut to main path and pass through ReLU activation
    x = layers.Add()([x, shortcut])
    x = layers.LeakyReLU()(x)

    return x

def create_resnet_model(input_shape=(128, 128, 3), num_classes=11):
    """ Creates a simplified ResNet-like model with reduced parameters """

    inputs = layers.Input(shape=input_shape)

    # Initial convolutional layer
    x = layers.Conv2D(16, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Residual blocks
    x = residual_block(x, filters=32, strides=(2, 2), use_projection=True)
    x = residual_block(x, filters=32, use_projection=False)

    x = residual_block(x, filters=64, strides=(2, 2), use_projection=True)
    x = residual_block(x, filters=64, use_projection=False)

    # Global average pooling and dense layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(32, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

In [ ]:
# Create the modified ResNet-like model
model = create_resnet_model()

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 64, 64, 16)           448       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 64, 64, 16)           64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 16)           0         ['batch_normalization[0][0

In [ ]:
# Compile the model
model.compile(optimizer='Nadam',
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

In [ ]:
# Define the learning rate schedule function
import math
def cosine_annealing(epoch, lr):
    epochs = 100  # Total number of epochs
    return 0.0001 + (0.1 - 0.0001) * (1 + math.cos(math.pi * epoch / epochs)) / 2

In [ ]:
from keras.callbacks import LearningRateScheduler

In [ ]:
# Learning rate scheduler
lr_scheduler = LearningRateScheduler(cosine_annealing)

In [ ]:
# Fit the model
history = model.fit(X_train, y_train, batch_size=64,
          epochs=50,
          validation_data=(X_validation, y_validation),
          callbacks=[lr_scheduler])

Epoch 1/50
117/117 [==============================] - 32s 82ms/step - loss: 2.3791 - accuracy: 0.1520 - val_loss: 9.3046 - val_accuracy: 0.1326 - lr: 0.1000
Epoch 2/50
117/117 [==============================] - 6s 50ms/step - loss: 2.2800 - accuracy: 0.1775 - val_loss: 3.6185 - val_accuracy: 0.1939 - lr: 0.1000
Epoch 3/50
117/117 [==============================] - 6s 52ms/step - loss: 2.2690 - accuracy: 0.1830 - val_loss: 2.9010 - val_accuracy: 0.1871 - lr: 0.0999
Epoch 4/50
117/117 [==============================] - 6s 50ms/step - loss: 2.2506 - accuracy: 0.1971 - val_loss: 2.6294 - val_accuracy: 0.2159 - lr: 0.0998
Epoch 5/50
117/117 [==============================] - 6s 51ms/step - loss: 2.2263 - accuracy: 0.2107 - val_loss: 2.5147 - val_accuracy: 0.2356 - lr: 0.0996
Epoch 6/50
117/117 [==============================] - 6s 51ms/step - loss: 2.2009 - accuracy: 0.2226 - val_loss: 2.4010 - val_accuracy: 0.2136 - lr: 0.0994
Epoch 7/50
117/117 [==============================] - 6s 51ms/s

In [ ]:
# Fit the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
          steps_per_epoch=len(X_train) // 64,
          epochs=50,
          validation_data=(X_validation, y_validation)
          )

In [ ]:
# save model
model.save('/content/CNN-Model-0.keras')

In [ ]:
loss, accuracy = model.evaluate(X_validation, y_validation)

# Print the evaluation results
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

55/55 [==============================] - 0s 5ms/step - loss: 1.0823 - accuracy: 0.6761
Test Loss: 1.0823
Test Accuracy: 0.6761


In [ ]:
# Predict the labels for the test images
predictions = model.predict(X_resized_test)

69/69 [==============================] - 0s 4ms/step


In [ ]:
# Convert predictions from one-hot encoding to class labels
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
# Create a DataFrame
df = pd.DataFrame({
    'id': index,
    'label': predicted_labels
})

In [ ]:
# Save to CSV
df.to_csv('AROOJ.csv', index=False)